In [28]:
from pandas import read_csv

In [29]:
dataframe = read_csv("../data/raw/houses_madrid.csv")

# Handle nulls values 
- We will drop the column if there is less than 40% of values

In [30]:
dataframe_observations = dataframe.shape[0]

for column in dataframe.columns:
    if dataframe[column].isnull().sum() >  dataframe_observations * 0.6:
        dataframe.drop(columns=[column], inplace=True)
        print(f"Drop column : {column}")

Drop column : sq_mt_useful
Drop column : n_floors
Drop column : sq_mt_allotment
Drop column : latitude
Drop column : longitude
Drop column : street_number
Drop column : portal
Drop column : door
Drop column : rent_price_by_area
Drop column : are_pets_allowed
Drop column : has_garden
Drop column : has_pool
Drop column : has_balcony
Drop column : has_storage_room
Drop column : is_furnished
Drop column : is_kitchen_equipped
Drop column : is_accessible
Drop column : has_green_zones
Drop column : has_private_parking
Drop column : has_public_parking
Drop column : is_parking_included_in_price
Drop column : parking_price
